# Graph Convolutional Network by Kipf and Welling

## Imports

In [1]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

import pickle
import numpy as np

import itertools

import Notebooks.performance as pf

## GNN Definition

In [2]:
class LinearModule(nn.Module):
    """The linear transformation part of the GCN layer"""
    def __init__(self, in_feats, out_feats, activation):
        super(LinearModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation # This is the activation function

    def forward(self, node):
        h = self.linear(node.data['h'])
        h = self.activation(h)
        return {'h' : h}

In [3]:
class GCN(nn.Module):
    """A GCN layer"""
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.apply_mod = LinearModule(in_feats, out_feats, activation)

    def forward(self, g, feature):
        g.ndata['h'] = feature
        g.update_all(message_func=fn.copy_src(src='h', out='m'), reduce_func=fn.sum(msg='m', out='h'))
        g.apply_nodes(func=self.apply_mod)
        return g.ndata.pop('h')

In [4]:
class Net(nn.Module):
    def __init__(self, infeats, hidden_size, outfeats):
        super(Net, self).__init__()
        self.gcn1 = GCN(infeats, hidden_size, F.relu)
        self.gcn2 = GCN(hidden_size, outfeats, lambda x: F.log_softmax(x,1))
        self.dropout = nn.Dropout(0.2)

    def forward(self, g, features):
        x = self.gcn1(g, features)
        #x = self.dropout(x)
        x = self.gcn2(g, x)
        return x

## Data Loading

In [14]:
from dgl.data import citation_graph as citegrh
import networkx as nx

data = citegrh.load_cora()
features = th.FloatTensor(data.features)
labels = th.LongTensor(data.labels)
mask = th.ByteTensor(data.train_mask)
g = data.graph

# add self loop
g.remove_edges_from(nx.selfloop_edges(g))
g = DGLGraph(g)
g.add_edges(g.nodes(), g.nodes())

## Select Training Set

In [6]:
percentage_train = 0.02

with open("data/cora_permutation1.pickle","rb") as f:
    perm1 = pickle.load(f)
mask = np.zeros(g.number_of_nodes())
mask[perm1[range(int(percentage_train*g.number_of_nodes()))]] = 1
mask = th.ByteTensor(mask)

## Training

In [7]:
loss_function = pf.perm_inv_loss(labels)

In [15]:
import time

net = Net(features.shape[1], 21, len(np.unique(labels)))
#print(net)

optimizer = th.optim.Adam(net.parameters(), lr=1e-2, weight_decay=1e-2)
net.train() # Set to training mode (use dropout)

dur = []
for epoch in range(500):
    if epoch >=3:
        t0 = time.time()

    # Compute loss for test nodes (only for validation, not used by optimizer)
    net.eval()
    prediction = net(g, features)
    train_rand=pf.rand_score(labels[mask].numpy(),np.argmax(prediction[mask].detach().numpy(), axis=1))
    validation_rand=pf.rand_score(labels[1-mask].numpy(),np.argmax(prediction[1-mask].detach().numpy(), axis=1))
    net.train()

    # Compute loss for train nodes
    logits = net(g, features)

    loss = loss_function.approximate_loss(logits,mask,nclasses=6)

    #loss = F.nll_loss(logits[mask], labels[mask])
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    print(f"Epoch {epoch:05d} | Loss {loss.item():.4f} | Train.Rand {train_rand:.4f} | Valid.Rand {validation_rand:.4f} | Time(s) {np.mean(dur):.4f}")

ch 00259 | Loss 1.8107 | Train.Rand 0.8763 | Valid.Rand 0.4647 | Time(s) 0.3600
Epoch 00260 | Loss 1.0993 | Train.Rand 0.8481 | Valid.Rand 0.4408 | Time(s) 0.3603
Epoch 00261 | Loss 1.1712 | Train.Rand 0.7991 | Valid.Rand 0.4068 | Time(s) 0.3601
Epoch 00262 | Loss 1.3080 | Train.Rand 0.7410 | Valid.Rand 0.3820 | Time(s) 0.3602
Epoch 00263 | Loss 1.1786 | Train.Rand 0.7289 | Valid.Rand 0.3750 | Time(s) 0.3605
Epoch 00264 | Loss 1.5865 | Train.Rand 0.7113 | Valid.Rand 0.3741 | Time(s) 0.3603
Epoch 00265 | Loss 1.5501 | Train.Rand 0.7144 | Valid.Rand 0.3665 | Time(s) 0.3604
Epoch 00266 | Loss 1.3335 | Train.Rand 0.7253 | Valid.Rand 0.3368 | Time(s) 0.3603
Epoch 00267 | Loss 1.2778 | Train.Rand 0.7301 | Valid.Rand 0.3437 | Time(s) 0.3602
Epoch 00268 | Loss 1.4221 | Train.Rand 0.7823 | Valid.Rand 0.3598 | Time(s) 0.3603
Epoch 00269 | Loss 1.4582 | Train.Rand 0.8691 | Valid.Rand 0.3678 | Time(s) 0.3602
Epoch 00270 | Loss 1.3362 | Train.Rand 0.8789 | Valid.Rand 0.3882 | Time(s) 0.3602
Epoch 0

In [20]:
# Visualise predictions
net.eval() # Set net to evaluation mode (deactivates dropout)
final_prediction = net(g, features).detach()
a = np.transpose(np.vstack([final_prediction[mask].numpy().argmax(axis=1),labels[mask].numpy()]))
a[a[:,0].argsort()][np.random.choice(range(a.shape[0]),size=10)]
# as can be seen, the net predicts other labels, but gets the clusters right :)

array([[0, 6],
       [0, 3],
       [2, 2],
       [2, 2],
       [0, 6],
       [6, 4],
       [4, 0],
       [4, 0],
       [0, 2],
       [0, 4]], dtype=int64)

## Evaluation

In [21]:
net.eval() # Set net to evaluation mode (deactivates dropout)
final_prediction = net(g, features).detach()
pf.performance_as_df(labels,final_prediction,mask)

,All,Train,Test
Mutual Information,0.200959,0.668763,0.193395
Rand-Index,0.092219,0.456485,0.088449
Variation of Information,2.655766,0.936472,2.667611
